# Custom model training for gesture recognition

Import the required libraries

In [2]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2024-10-21 17:27:33.203143: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 17:27:33.248327: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 17:27:33.250692: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 17:27:34.485007: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Load the list of gesture form the `gesture_list.csv` file

In [3]:
# Define which gestures should be detected by the model
gesture_list = np.loadtxt("gesture_list.csv", delimiter=',', dtype=str)
print(gesture_list)
num_gestures = len(gesture_list)

['none' 'open' 'fist' 'index' 'L']


Load the dataset from the file

In [4]:

dataset_filepath = "dataset.csv"
x_data = np.loadtxt(dataset_filepath, delimiter=',', dtype=np.float32, usecols=list(range(1, (21*3)+1)))
y_data_labels = np.loadtxt(dataset_filepath, delimiter=',', dtype=np.str_, usecols=(0))

Define the ID of each gesture. The ID coincides with the row number in the file

In [5]:
y_data = np.full((y_data_labels.shape[0]), 0)
for i, label in enumerate(y_data_labels):
    for j, gesture in enumerate(gesture_list):
        if label == gesture and gesture in gesture_list:
            # "Substitute" the label with the gesture id
            y_data[i] = j

Split the dataset into training and validation sets

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.75)
print(f"Y training set: {y_train}")

Y training set: [2 3 2 3 2 4 4 1 3 2 4 4 4 3 1 3 4 4 3 1 3 4 1 1 3 4 4 3 4 4 2 1 2 3 3 4 3
 1 1 2 3 3 3 2 4 3 3 3 4 4 2 2 2 4 2 3 1 1 2 3 4 2 2 4 3 1 3 4 4 3 2 3 2 2
 1 3 3 4 2 4 1 2 1 4 1 1 4 2 3 4 4 2 4 2 3 4 2 2 4 4 3 2 3 1 3 2 2 4 1 1 3
 1 4 4 2 3 3 2 1 2 1 2 1 1 3 1 4 1 1 4 3 2 2 2 3 1 2 1 2 3 4 1 2 2 3 2 2 1
 4 2 2 2 3 3 2 2 2 4 1 4 1 4 1 1 3 3 1 1 1 4 1 4 2 4 3 2 4 4 1 4 1 3 2 3 1
 2 4 4 2 1 4 1 2 3 2 4 1 1 1 4 2 3 1 4 4 2 2 1 3 1 4 3 3 3 2 4 2 2 1 1 3 3
 2 1 2 3 2 3 4 2 2 2 2 1 3 2 4 1 3 4 3 4 4 1 4 2 2 3 4 4 4 2 2 1 4 1 1 3 1
 2 4 3 4 4 1 4 2 2 2 3 2 1 1 1 4 4 4 4 1 4 4 4 2 1 1 1 4 1 4 3 3 1 3 1 4 2
 4 3 1 2]


Define the model architechture

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 3, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(num_gestures, activation='softmax')
])
model.summary()

# Path where to save the model
model_save_path = "model/gesture_classifier.hdf5"
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 63)                0         
                                                                 
 dense (Dense)               (None, 20)                1280      
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dropout_2 (Dropout)         (None, 20)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
 dense_3 (Dense)             (None, 5)                 5

Compile the model

In [8]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Training step

In [9]:
model.fit(
    x_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(x_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
1/3 [=========>....................] - ETA: 1s - loss: 1.6129 - accuracy: 0.2812
Epoch 1: saving model to model/gesture_classifier.hdf5
3/3 [==============================] - 1s 109ms/step - loss: 1.6079 - accuracy: 0.2800 - val_loss: 1.5957 - val_accuracy: 0.3900
Epoch 2/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.6004 - accuracy: 0.2500
Epoch 2: saving model to model/gesture_classifier.hdf5
3/3 [==============================] - 0s 24ms/step - loss: 1.5976 - accuracy: 0.2967 - val_loss: 1.5915 - val_accuracy: 0.3400
Epoch 3/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.5994 - accuracy: 0.2812
Epoch 3: saving model to model/gesture_classifier.hdf5
3/3 [==============================] - 0s 23ms/step - loss: 1.5946 - accuracy: 0.2900 - val_loss: 1.5879 - val_accuracy: 0.2300
Epoch 4/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.5948 - accuracy: 0.2734
Epoch 4: saving model to model/gesture_classifier.hdf5
3/3 [==============

/home/eli/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 5: saving model to model/gesture_classifier.hdf5
3/3 [==============================] - 0s 22ms/step - loss: 1.5843 - accuracy: 0.3100 - val_loss: 1.5803 - val_accuracy: 0.2000
Epoch 6/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.5783 - accuracy: 0.2891
Epoch 6: saving model to model/gesture_classifier.hdf5
3/3 [==============================] - 0s 23ms/step - loss: 1.5808 - accuracy: 0.3000 - val_loss: 1.5754 - val_accuracy: 0.2000
Epoch 7/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.5794 - accuracy: 0.2969
Epoch 7: saving model to model/gesture_classifier.hdf5
3/3 [==============================] - 0s 43ms/step - loss: 1.5757 - accuracy: 0.3033 - val_loss: 1.5701 - val_accuracy: 0.2000
Epoch 8/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.5690 - accuracy: 0.3047
Epoch 8: saving model to model/gesture_classifier.hdf5
3/3 [==============================] - 0s 22ms/step - loss: 1.5697 - accuracy: 0.2967 - val_loss: 1.5644 - val_ac

Validation step

In [10]:
val_loss, val_acc = model.evaluate(x_test, y_test, batch_size=128)

1/1 [==============================] - 0s 24ms/step - loss: 0.0640 - accuracy: 0.9900


Save the model with inference optimizations

In [ ]:
model.save(model_save_path, include_optimizer=False)